In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from fake_useragent import UserAgent

import pandas as pd
import requests
import time
import json
import re
import warnings

from tqdm.notebook import tqdm

from slimit import ast
from slimit.parser import Parser
from slimit.visitors import nodevisitor

warnings.filterwarnings(action='ignore')
headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36'}

In [2]:
NATE_URL = 'https://www.nate.com/?f=natesvc'
ZUM_URL = 'https://zum.com/#!/home'
GOOGLE_URL = 'https://trends.google.co.kr/trends/trendingsearches/daily/rss?geo=KR'
BLACK_URL = 'https://blackkiwi.net/service/trend'

In [3]:
Status = True
while Status:
    ZUM_REQ = requests.get(ZUM_URL, headers=headers)
    ZUM_SOUP = BeautifulSoup(ZUM_REQ.text, 'html.parser')

    ZUM_TREND = ZUM_SOUP.find_all('span', {'class':'keyword d_keyword'})[:20]
    zum_data = []

    for i in range(0, len(ZUM_TREND), 2):
        Keyword = ZUM_TREND[i].text

        zum_list = {'Keyword (zum)':Keyword}
        zum_data.append(zum_list)

    zum_df = pd.DataFrame(zum_data, columns=['Keyword (zum)'])
    zum_df.index = pd.RangeIndex(start=1, stop=(len(zum_df)+1))

    if zum_df.iloc[0][0] == zum_df.iloc[1][0]:
        continue
    else:
        Status = False

In [4]:
Status = True
while Status:
    nate_data = []
    
    for i in range(2):
        NATE_REQ = requests.get(NATE_URL, headers=headers)
        NATE_SOUP = BeautifulSoup(NATE_REQ.text, 'html.parser')

        for j in range(0, 5):
            Keyword = NATE_SOUP.find_all('span', {'class':'txt_rank'})[j].text
            Rank = int(NATE_SOUP.find_all('span', {'class':'num_rank'})[j].text)

            nate_list = {'Rank':Rank, 'Keyword (nate)':Keyword}

            nate_data.append(nate_list)
            
    nate_df = pd.DataFrame(nate_data, columns=['Rank', 'Keyword (nate)'])
    nate_df.sort_values("Rank", axis = 0, ascending = True, inplace = True)
    nate_df = nate_df.drop(['Rank'], axis='columns')
    nate_df.index = pd.RangeIndex(start=1, stop=(len(nate_df)+1))
    
    if nate_df.iloc[0][0] == nate_df.iloc[1][0]:
        continue
    else:
        Status = False

In [5]:
GOOGLE_REQ = requests.get(GOOGLE_URL, headers=headers)
GOOGLE_SOUP = BeautifulSoup(GOOGLE_REQ.text, 'html.parser')

google_data = []

for i in range(1, 11):
    Rank = int(i)
    Keyword = GOOGLE_SOUP.find_all('title')[i].text
    
    google_list = {'순위':Rank, 'Keyword (google)':Keyword}
    
    google_data.append(google_list)

google_df = pd.DataFrame(google_data, columns=['순위', 'Keyword (google)'])
google_df.sort_values("순위", axis = 0, ascending = True, inplace = True)
google_df = google_df.drop(['순위'], axis='columns')
google_df.index = pd.RangeIndex(start=1, stop=len(google_df)+1)

In [6]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
DV = webdriver.Chrome(ChromeDriverManager().install(), options=options)

DV.get(BLACK_URL)

BLACK_SOUP = BeautifulSoup(DV.page_source, 'html.parser')
DV.close()

kiwi_data = []
cnt = 0
for i in BLACK_SOUP.find_all('a', {'class':'style__ExternalLink-sc-173w3uh-0 fnazVE'}):
    Rank = cnt+1
    Keyword = i.text
    
    kiwi_list = {'순위':Rank, 'Keyword (black kiwi)':Keyword}
    
    kiwi_data.append(kiwi_list)
    
kiwi_df = pd.DataFrame(kiwi_data, columns=['순위', 'Keyword (black kiwi)'])
kiwi_df.sort_values("순위", axis = 0, ascending = True, inplace = True)
kiwi_df = kiwi_df.drop(['순위'], axis='columns')
kiwi_df.index = pd.RangeIndex(start=1, stop=len(kiwi_df)+1)

[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/89.0.4389.23/chromedriver_mac64.zip


[WDM] - Driver has been saved in cache [/Users/gimgyeonghwan/.wdm/drivers/chromedriver/mac64/89.0.4389.23]


In [7]:
result = pd.concat([nate_df, zum_df, google_df, kiwi_df], axis=1)

result

,Keyword (nate),Keyword (zum),Keyword (google),Keyword (black kiwi)
1,766억 기부 이수영,기성용,이수영 회장,에릭
2,김동완,박우식,오세훈,현주엽
3,고스트 닥터,비트코인,디카르고,오세훈
4,2월 취업자 47만 3000명,이수영회장,카카오 이베이,로제
5,김하성 시범경기 첫 신고,임영웅,황사,황사
6,현주엽,현주엽,리플,디카르고
7,홍남기 환골탈태 방안 발표,디카르고,구미 친모,리플
8,AZ백신 근거 없어,공시지가조회,윤여정,EV6
9,경로를 이탈하였습니다,경로를이탈하였습니다,공시지가,카카오 이베이
10,박혜수,롯데캐슬드메르,현주엽,오케이 광자매


In [9]:
NAVER_NEWS_URL = 'https://news.naver.com/main/ranking/popularDay.nhn'
NAVER_REQ = requests.get(NAVER_NEWS_URL, headers=headers)
NAVER_SOUP = BeautifulSoup(NAVER_REQ.text, 'html.parser')

contents = NAVER_SOUP.find_all('div', {'class':'rankingnews_box'})

naver_news_list = []

for i in range(0, len(contents)):
    for j in range(0, len(contents[i]('em'))):
        rank = contents[i]('em')[j].text
        news = contents[i]('strong')[0].text
        title = contents[i]('a', {'class':'list_title nclicks(\'RBP.rnknws\')'})[j].text
        list_time = contents[i]('span', {'class':'list_time'})[j].text
        url = 'https://news.naver.com' + contents[i]('a', {'class':'list_title nclicks(\'RBP.rnknws\')'})[j]['href']
        
        naver_news_data = {'순위':rank, '언론사':news, '제목':title, '작성시간':list_time, '주소':url}
        
        naver_news_list.append(naver_news_data)
        
news_df = pd.DataFrame(naver_news_list, columns=['순위', '언론사', '제목', '작성시간', '주소'])

news_df[news_df['순위']=='1']

,순위,언론사,제목,작성시간,주소
0,1,YTN,"""불 났다""...유흥업소의 화재신고 뒤에 숨겨진 '꼼수'",5시간전,https://news.naver.com/main/ranking/read.nhn?m...
5,1,중앙일보,"[단독]접종 8일뒤 숨진 60대 다리에 혈전…""연관성 확인 안돼""",5시간전,https://news.naver.com/main/ranking/read.nhn?m...
10,1,국민일보,“성기 모양 기억”… 기성용 성폭행 의혹 ‘PD수첩’ 증언,3시간전,https://news.naver.com/main/ranking/read.nhn?m...
15,1,JTBC,'LH 땅 투기 의혹' 결국 특검으로…경찰은 '당혹·불만',2시간전,https://news.naver.com/main/ranking/read.nhn?m...
20,1,매일신문,20대 여대생 성폭행+고시원에 한달 감금한 30대 남성,13시간전,https://news.naver.com/main/ranking/read.nhn?m...
...,...,...,...,...,...
333,1,이데일리,"“건장했던 20대 아들, 온라인 게임서 말다툼 후 살해돼”",3시간전,https://news.naver.com/main/ranking/read.nhn?m...
338,1,주간동아,"미얀마 군부 쿠데타, 中 일석삼조 호재",23시간전,https://news.naver.com/main/ranking/read.nhn?m...
343,1,중앙SUNDAY,삼각 지붕 교실에 정원 19개 ‘숨구멍’…고정관념 깬 신길중,4일전,https://news.naver.com/main/ranking/read.nhn?m...
348,1,기자협회보,기협만평 2021.3.17,10시간전,https://news.naver.com/main/ranking/read.nhn?m...
